In [6]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import tqdm

## Use perplexity to do fact checking

HuggingFace Reference: https://huggingface.co/docs/transformers/perplexity

Paper: Lee, Nayeon, et al. "Towards Few-Shot Fact-Checking via Perplexity." NAACL (2021)

In [2]:
device = 'cpu'
model_id = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [12]:
evidence = "I wake up at 6 AM in the morning."
claim1 = "Then I ate dinner and went doing exercise."
claim2 = "Then I put on my cloth and brushed my teeth."
claim3 = "Marry is a sexy strip teaser."
evidence_inp = tokenizer(evidence, return_tensors='pt')
claim_inp = tokenizer(claim1, return_tensors='pt')
pair_inp = torch.cat([evidence_inp.input_ids, claim_inp.input_ids], axis=-1)

In [13]:
# option 2
tgt_len = claim_inp.input_ids.size(1)
input_ids = pair_inp.to(device)
target_ids = input_ids.clone()
target_ids[:,:-tgt_len] = -100

with torch.no_grad():
    outputs = model(input_ids, labels=target_ids)
    neg_log_likelihood = outputs[0] * tgt_len
print(neg_log_likelihood)

tensor(45.0882)


In [15]:
type(neg_log_likelihood.tolist())

float

In [7]:
# option 1
stride = 1
nlls = []
context_len = evidence_inp.input_ids.size(1)

for i in tqdm(range(context_len, pair_inp.size(1), stride)):
    begin_loc = 0
    end_loc = i+1
    input_ids = pair_inp[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:i] = -100
    #import pdb;pdb.set_trace()
    
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0]
    nlls.append(neg_log_likelihood)
    
print(sum(nlls))

TypeError: 'module' object is not callable